In [1]:
from azureml.core import Workspace

# Load existing workspace from the config file info.
ws = Workspace.from_config()

Found the config file in: /data/home/jomit/notebooks/Flex Workshop/aml_config/config.json


In [2]:
from azureml.core.model import Model

model = Model.register(model_path = "model_Linear.pickle",
                        model_name = "model_Linear",
                        tags = {"data": "EngineData", "type": "classification"},
                        description = "Predictive Maintenance",
                        workspace = ws)

Registering model model_Linear


In [3]:
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
#myenv.add_conda_package("sklearn")
#myenv.add_conda_package("keras")
#myenv.add_conda_package("tensorflow")
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [4]:
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = "Score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  dependencies = ['min_max_scaler.pickle','model_Linear.pickle'],
                                                  description = "",
                                                  tags = {"type": "classification"})

In [5]:
image = ContainerImage.create(name = "pdmimage1",
                                # this is the model object
                                models = [model],
                                image_config = image_config,
                                workspace = ws)

# Wait for the create process to complete
image.wait_for_creation(show_output = True)

Creating image
Running........................................................
SucceededImage creation operation finished for image pdmimage1:1, operation "Succeeded"


In [6]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'hackfest-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating......................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


In [7]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='pdm-service-1'
# Deploy from image
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                            name = aks_service_name,
                                            image = image,
                                            deployment_config = aks_config,
                                            deployment_target = aks_target)
# Wait for the deployment to complete
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running................................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy


In [8]:
import Score
import json

test_df = Score.getTestInput()  # [?, 26]
test_df = test_df[test_df['id']==20] # ? filtered by id
test_df = test_df.tail(5)

test_json = test_df.to_json(orient='split')

#test_sample = json.dumps({'data': [
#    [1,2,3,4,5,6,7,8,9,10], 
#    [10,9,8,7,6,5,4,3,2,1]
#]})
test_sample = bytes(test_json,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

1
